# Apartments

In [1]:
import rdata
import pandas as pd
import numpy as np
import category_encoders as ce
import copy
import random
import warnings 
warnings.filterwarnings('ignore')
np.random.seed(666) 

C:\Users\hp\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\hp\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# ładowanie apartments
parse = rdata.parser.parse_file("apartments.rda")
convert = rdata.conversion.convert(parse)
apartments = convert["apartments"]

In [3]:
apartments.describe()

,m2.price,construction.year,surface,floor,no.rooms
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3487.019000,1964.823000,85.590000,5.623000,3.360000
std,906.691651,25.831511,37.874799,2.899876,1.381415
min,1607.000000,1920.000000,20.000000,1.000000,1.000000
25%,2857.000000,1943.000000,53.000000,3.000000,2.000000
50%,3386.000000,1965.000000,85.500000,6.000000,3.000000
75%,4018.250000,1988.000000,118.000000,8.000000,4.000000
max,6595.000000,2010.000000,150.000000,10.000000,6.000000


In [4]:
apartments.district.unique()

[Srodmiescie, Bielany, Praga, Ochota, Mokotow, Ursus, Zoliborz, Wola, Bemowo, Ursynow]
Categories (10, object): [Srodmiescie, Bielany, Praga, Ochota, ..., Zoliborz, Wola, Bemowo, Ursynow]

In [5]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   m2.price           1000 non-null   float64 
 1   construction.year  1000 non-null   float64 
 2   surface            1000 non-null   float64 
 3   floor              1000 non-null   int64   
 4   no.rooms           1000 non-null   float64 
 5   district           1000 non-null   category
dtypes: category(1), float64(4), int64(1)
memory usage: 40.6 KB


Naszą zmienną celu będzie m2.price. Jak widać nie ma braków więc nie trzeba niczego uzupełniać. Jedyne co zmienimy w datasecie to wykorzystamy encoding żeby pozbyć się stringów.

In [6]:
ohe = ce.OneHotEncoder(cols=['district'], use_cat_names=True)
ohe.fit(apartments)
apartments = ohe.transform(apartments)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(apartments.drop('m2.price', axis=1), apartments['m2.price'])

## Model bez strojenia

In [8]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
svr = SVR()
svr.fit(X_train, y_train)
mean_squared_error(y_test, svr.predict(X_test))

703346.9195528338

Jak widać bez strojenia jest na razie słabo.

## Strojenie hiperparametrów

In [9]:
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV

In [10]:
C = sp_uniform(scale=10000)
gamma = sp_uniform(scale=1)
degree = sp_uniform(scale = 1000)

params1 = {'kernel': ['rbf'],
              'C':C, 
              'gamma': gamma,
              'degree': degree
 }

In [11]:
random_search = RandomizedSearchCV(estimator=SVR(epsilon = 0.01), 
                            param_distributions=params1,
                            n_jobs=-1, 
                            verbose=2,
                            cv=4,
                            n_iter=5000)

In [12]:
random_result = random_search.fit(X_train, y_train)
random_result.best_params_

Fitting 4 folds for each of 5000 candidates, totalling 20000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 562 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 1858 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 2748 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 3802 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 5016 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 6394 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 7932 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 9634 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 11496 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 13522 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 15708 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 18058 tasks  

{'C': 9698.754877349842,
 'degree': 77.91114804386623,
 'gamma': 0.002966783860547384,
 'kernel': 'rbf'}

In [13]:
C2 = random_result.best_params_['C']
gamma2 = random_result.best_params_['gamma']
degree2 = random_result.best_params_['degree']

In [14]:
svr.set_params(C = C2, gamma = gamma2, degree = degree2)
svr.fit(X_train, y_train)

SVR(C=9698.754877349842, cache_size=200, coef0=0.0, degree=77.91114804386623,
    epsilon=0.1, gamma=0.002966783860547384, kernel='rbf', max_iter=-1,
    shrinking=True, tol=0.001, verbose=False)

In [15]:
mean_squared_error(y_test, svr.predict(X_test))

300428.8005255286

Nadal nie jest idealnie ale wyniki są zdecydowanie lepsze.

# Pokemon

In [16]:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


Tutaj już mamy nulle w 'Type 2' więc zmienimy na string. 

In [17]:
pokemon = pokemon.replace(np.nan, 'None', regex=True)
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,None,309,39,52,43,60,50,65,1,False


Pozbędziemy się też pierwszych dwóch kolumn bo są one niepotrzebne. 'Type 1' i 'Type 2' zaenkodujemy, a za zmienną celu uznamy 'Legendary'.

In [18]:
pokemon2 = pokemon.loc[:,'Type 1':'Legendary']
ohe = ce.OneHotEncoder(cols=['Type 1', 'Type 2'], use_cat_names=True)
ohe.fit(pokemon2)
pokemon2 = ohe.transform(pokemon2)

In [19]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(pokemon2.drop('Legendary', axis=1), pokemon2['Legendary'], test_size=0.4)

## Model bez strojenia

In [20]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model = svm.SVC()
model.fit(X_train2, y_train2)
accuracy_score(y_test2, model.predict(X_test2))

0.945

In [21]:
print(confusion_matrix(y_test2, model.predict(X_test2)))

[[364   8]
 [ 14  14]]


Jak widać mamy tyle samo False Negatives co True Postives. Jest też kilka False Positives ale przynajmniej jest ich mniej niż True Positives.

## Strojenie hiperparametrów

In [22]:
C = sp_uniform(scale=10000)
gamma = sp_uniform(scale=1)
degree = sp_uniform(scale = 1000)

params2 = {'kernel': ['rbf'],
              'C':C, 
              'gamma': gamma,
              'degree': degree
 }

In [23]:
random_search2 = RandomizedSearchCV(model, 
                                    param_distributions=params2,
                                    n_jobs=-1, 
                                    verbose=2,
                                    cv=4,
                                    n_iter=5000)
random_search2.fit(X_train2, y_train2)

Fitting 4 folds for each of 5000 candidates, totalling 20000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 696 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 2320 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 4584 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 7504 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 11064 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 15280 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 19961 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 20000 out of 20000 | elapsed:   49.4s finished


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=5000, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021994737198>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000219947376A0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021994737160>,
                                        'kernel': ['rbf']},
                   pre_dispatch=

In [24]:
print(f"Score: {random_search2.best_score_}")
random_search2.best_params_

Score: 0.9424999999999999


{'C': 7506.092253647926,
 'degree': 6.4207359109555995,
 'gamma': 7.526853552863866e-05,
 'kernel': 'rbf'}

In [25]:
C3 = random_search2.best_params_['C']
gamma3 = random_search2.best_params_['gamma']
degree3 = random_search2.best_params_['degree']

In [26]:
model.set_params(C=C3, gamma=gamma3, degree=degree3)
model.fit(X_train2, y_train2)

SVC(C=7506.092253647926, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=6.4207359109555995,
    gamma=7.526853552863866e-05, kernel='rbf', max_iter=-1, probability=False,
    random_state=None, shrinking=True, tol=0.001, verbose=False)

In [27]:
print(confusion_matrix(y_test2, model.predict(X_test2)))

[[358  14]
 [  8  20]]


Stosunek False Negatives do True Positives zmienił się z 1:1 do 1:2.5, ale za to zwiększyła się trochę liczba False Positives. Nie jest to duża poprawa, a możnaby nawet powiedzieć że znikoma.